In [1]:
import geopandas
import pandas as pd

In [2]:
def from_df_to_geodf(df, long_col, lat_col):
    
    gdf = geopandas.GeoDataFrame(
        df, geometry=geopandas.points_from_xy(df[lat_col], 
                                              df[long_col]))
    
    return gdf

def mapas_geosampa(path):
    
    map_geodf = geopandas.read_file(path)
    map_geodf.crs = {'init' :'epsg:31983'}
    map_geodf = map_geodf.to_crs(epsg = 4326)
    
    return map_geodf

def geodf_dados_participe(path_file):
    
    df = pd.read_excel(path_file)
    df = from_df_to_geodf(df, 'latitude', 'longitude')
    df.crs = {'init' :'epsg:4326'}
    df = df.to_crs(epsg = 4326)
    
    return df

def join_contains(mapa_geosampa, df_participe):
    
    #faz um "contém" - serve para relaçao poligono -> ponto
    s_join = geopandas.sjoin(mapa_geosampa, 
                             df_participe, 
                             how="right", op='contains')
    
    return s_join

def pegar_municipio(df_participe):
    
    #vai carregar toda vez mas é rapidinho e assim encapsula
    #nao vou precisar desse geodf depois
    municipios = mapas_geosampa('mapas_geosampa/SIRGAS_SHP_limites_municipais/'
                             'SIRGAS_limites_municipais.shp')
    join_m = join_contains(municipios, df_participe)
    
    df_participe['municipio_geosampa'] = join_m['municipio']
    
def pegar_distrito_e_subs(df_participe, distritos_geosampa):
    
    df_sampa = df_participe[df_participe['municipio_geosampa']=='SÃO PAULO'].copy()
    cols_interesse = {'ds_cd_sub' : 'subprefeitura_codigo',
                      'ds_sigla' : 'distrito_sigla', 
                      'ds_subpref' : 'subprefeitura', 
                      'ds_nome' : 'distrito_nome', 
                      'ds_codigo' : 'distrito_codigo'}
    
    info_distritos = join_contains(distritos_geosampa, df_participe)
    info_distritos = info_distritos[cols_interesse.keys()].reset_index()
    info_distritos.rename(cols_interesse, axis = 1, inplace = True)
    
    return pd.merge(df_participe.reset_index(), info_distritos, how = 'left', 
                    right_on = 'index_right', left_on = 'index')

In [3]:
def toilete_dados_final(df, nom_df):
    
    cols_to_drop = {
        'contrib' : [
            'index',
            'Unnamed: 0',
            'Unnamed: 0.1',	
            'Endereço',
            'index_right',
            'geometry'
        ],
        'prop' : [
            'index',
            'Unnamed: 0',
            'Unnamed: 0.1',
            'Unnamed: 18',
            'Unnamed: 19',
            'Unnamed: 20',
            'Unnamed: 21',
            'Unnamed: 22',
            'Endereco',
            'index_right',
            'geometry'
        ]
    }
    
    return df.drop(cols_to_drop[nom_df], axis = 1).copy()

In [4]:
distritos = mapas_geosampa('mapas_geosampa/SIRGAS_SHP_distrito/'
                           'SIRGAS_SHP_distrito_polygon.shp')

contrib_georref = geodf_dados_participe('../Dados_gerados/contrib_georref_final.xlsx')
prop_georref = geodf_dados_participe('../Dados_gerados/prop_georref_final.xlsx')

In [5]:
pegar_municipio(prop_georref)
pegar_municipio(contrib_georref)

C:\Users\h-pgy\Anaconda3\lib\site-packages\numpy\lib\function_base.py:2192: RuntimeWarning: invalid value encountered in ? (vectorized)
  outputs = ufunc(*inputs)
C:\Users\h-pgy\Anaconda3\lib\site-packages\numpy\lib\function_base.py:2192: RuntimeWarning: invalid value encountered in ? (vectorized)
  outputs = ufunc(*inputs)


In [6]:
contrib_georref_geosampa = pegar_distrito_e_subs(contrib_georref, distritos)
prop_georref_geosampa = pegar_distrito_e_subs(prop_georref, distritos)

#só para certificar que o merge tá ok
assert len(contrib_georref_geosampa) == len(contrib_georref)
assert len(prop_georref_geosampa) == len(prop_georref)

C:\Users\h-pgy\Anaconda3\lib\site-packages\numpy\lib\function_base.py:2192: RuntimeWarning: invalid value encountered in ? (vectorized)
  outputs = ufunc(*inputs)
C:\Users\h-pgy\Anaconda3\lib\site-packages\numpy\lib\function_base.py:2192: RuntimeWarning: invalid value encountered in ? (vectorized)
  outputs = ufunc(*inputs)


In [7]:
contrib_georref_final = toilete_dados_final(contrib_georref_geosampa, 'contrib')
prop_georref_final = toilete_dados_final(prop_georref_geosampa, 'prop')

In [8]:
path_dados = '../Dados_gerados/'
prop_georref_final.to_excel(path_dados + 'Propostas_georreferenciadas_final.xlsx')
contrib_georref_final.to_excel(path_dados + 'Revisoes_georreferenciadas_final.xlsx')

In [9]:
#salvar em csv também para pessoal analisar no qgis
prop_georref_final.to_csv(path_dados  + 'propostas_p+_georreferenciadas.csv', 
                          sep = ';', encoding = 'utf-8')

contrib_georref_final.to_csv(path_dados  + 'revisoes_p+_georreferenciadas.csv', 
                          sep = ';', encoding = 'utf-8')